
# <div style="text-align: center;">  Підготовка та аналіз даних  </div> #

# <div style="text-align: center;"> _Лабораторна робота №2_ </div> #



## <div style="text-align: right;"> _ФБ-24 Бадрак Іван_ </div> ##





### 1. Створити env в якому будуть встановлені всі необхідні бібліотеки та налаштування для данної лабораторної роботи. ###
### Для цього завантажив Anaconda Navigator, створив новий env та проімпортував бібліотеки, які необхідні для виконання лабораторної роботи.
### 2. Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження; ###

In [53]:
from datetime import datetime
import os
import urllib

url="https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2023&type=Mean"
datentime = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")

def DataDownload(url, datentime):
    if not os.path.exists('DataCSV'):
        os.makedirs('DataCSV')

    for regionid in range(1, 28):
        file_exists = False
        for file in os.listdir('DataCSV'):   
            if file.startswith(f'NOAA_ID{regionid}'):
                print (f'Файл NOAA_ID{regionid} вже завантажений')
                file_exists = True
                break
        if file_exists:
            continue
        URL = url.format(regionid)
        filename = f"NOAA_ID{regionid}_{datentime}.csv"
        filepath = os.path.join("DataCSV", filename)
        try:
            urllib.request.urlretrieve(URL, filepath)
            print (f"{filename} успішно завантажений")
        except Exception as exc:
            print(f"Відбулася помилка при скачуванні файлу для номеру області {regionid}: {str(exc)}")

    return 'Файли завантажені успішно'
    

DataDownload(url, datentime)

NOAA_ID1_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID2_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID3_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID4_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID5_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID6_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID7_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID8_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID9_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID10_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID11_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID12_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID13_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID14_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID15_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID16_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID17_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID18_26-03-2024-13-57-25.csv успішно завантажений
NOAA_ID19_26-03-202

'Файли завантажені успішно'

### 3. Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли; ###

In [69]:
import pandas as pd
import os

def CleanData(Path):
    list = []

    if not os.path.exists(Path) or not os.path.isdir(Path):
        print ('Вказаного каталогу не існує або не є каталогом')
    else:
        noaafiles = os.listdir(Path)
        for file in noaafiles:

            if file.endswith('.csv'):
                
                filepath = os.path.join(Path, file)   # Взяти індекси областей з назв файлів за допомогою split, зробити перевірку на NaN (вивести скільки їх в датафреймі)
                df = pd.read_csv(filepath, index_col=False, header=1)  
                df['ID'] = int(file.split('_')[1][2:]) 
                list.append(df)
                
        df = pd.concat(list).drop_duplicates().reset_index(drop=True)

    # print (df) # before
    df = df.rename(columns={' VHI<br>': 'VHI'})

    # print (df.head())
    df['year'].replace({'<tt><pre>1982': '1982'}, inplace=True)
    df = df.drop(df.loc[df['VHI'] == -1].index)

    # df = df.drop(58994)

    # nan_count = df.isna().sum().sum()
    # print (nan_count)
    df = df.dropna()

    # print (df.dtypes)
    df['year'] = pd.to_numeric(df['year'], errors='coerce')

    # print (f'{df.head()}\n{df.dtypes}\n{df.tail()}')
    # print (df)
    return df

path = 'DataCSV'
clean_df = CleanData(path)
print (clean_df)

year     object
week    float64
 SMN    float64
SMT     float64
VCI     float64
TCI     float64
VHI     float64
ID        int64
dtype: object
       year  week    SMN     SMT    VCI    TCI    VHI  ID
0      1982   1.0  0.059  258.24  51.11  48.78  49.95  10
1      1982   2.0  0.063  261.53  55.89  38.20  47.04  10
2      1982   3.0  0.063  263.45  57.30  32.69  44.99  10
3      1982   4.0  0.061  265.10  53.96  28.62  41.29  10
4      1982   5.0  0.058  266.42  46.87  28.57  37.72  10
...     ...   ...    ...     ...    ...    ...    ...  ..
58989  2023  48.0  0.101  277.36  37.50  15.86  26.68   9
58990  2023  49.0  0.097  275.73  38.12  16.77  27.45   9
58991  2023  50.0  0.097  275.42  40.98  13.16  27.07   9
58992  2023  51.0  0.097  274.98  43.98  11.14  27.56   9
58993  2023  52.0  0.098  274.62  47.00  10.39  28.70   9

[57618 rows x 8 columns]


### 4. Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на наступні: ###

In [70]:
import pandas as pd
import os

clean_df = CleanData(path)

def ChangeIndex (df):
    new_index = {
        1: 22,
        2: 24,
        3: 23,
        4: 25,
        5: 3,
        6: 4,
        7: 8,
        8: 19,
        9: 20,
        10: 21,
        11: 9,
        12: 9,
        13: 10,
        14: 11,
        15: 12,
        16: 13,
        17: 14,
        18: 15,
        19: 16,
        20: 25,
        21: 17,
        22: 18,
        23: 6,
        24: 1,
        25: 2,
        26: 7,
        27: 5,
    }

    df['ID'].replace( new_index, inplace=True)
    # print (df)
    return df

change_df = ChangeIndex(clean_df)
print (change_df)


year     object
week    float64
 SMN    float64
SMT     float64
VCI     float64
TCI     float64
VHI     float64
ID        int64
dtype: object
       year  week    SMN     SMT    VCI    TCI    VHI  ID
0      1982   1.0  0.059  258.24  51.11  48.78  49.95  21
1      1982   2.0  0.063  261.53  55.89  38.20  47.04  21
2      1982   3.0  0.063  263.45  57.30  32.69  44.99  21
3      1982   4.0  0.061  265.10  53.96  28.62  41.29  21
4      1982   5.0  0.058  266.42  46.87  28.57  37.72  21
...     ...   ...    ...     ...    ...    ...    ...  ..
58989  2023  48.0  0.101  277.36  37.50  15.86  26.68  20
58990  2023  49.0  0.097  275.73  38.12  16.77  27.45  20
58991  2023  50.0  0.097  275.42  40.98  13.16  27.07  20
58992  2023  51.0  0.097  274.98  43.98  11.14  27.56  20
58993  2023  52.0  0.098  274.62  47.00  10.39  28.70  20

[57618 rows x 8 columns]


### 5. Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу): 

### Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);
### Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25);
### Аналогічно для помірних посух

In [71]:
import pandas as pd
import os

change_df = ChangeIndex(clean_df)

# Ряд VHI для області за вказаний рік, пошук екстремумів (min та max) 
def ExtremesByYear(df, year, regionid):
    min = df[(df['year'] == year) & (df['ID'] == regionid)]['VHI'].min()
    max = df[(df['year'] == year) & (df['ID'] == regionid)]['VHI'].max()
    print (f'Мінімальне значення VHI за {year} рік для області під номером {regionid}: {min}\nМаксимальне значення VHI за {year} рік для області під номером {regionid}: {max}')

# Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися
# більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25);
def YearswExtremedr(df, first_year, second_year):
    all = df[(df['year'] >= first_year) & (df['year'] <= second_year)]['week'].count()
    required = df[(df['year'] >= first_year) & (df['year'] <= second_year) & (df['VHI'] < 15)]['week'].count()
    percent = (required/all)*100
    print (f'За період з {first_year} по {second_year} було {required} тижнів з екстримальними посухами.\nЦе дорівнює {percent}% від усієї кількості тижнів цього діапазону')

# Аналогічно для помірних посух
def YearswModeratedr(df, first_year, second_year):   # Помірною посухою будемо вважати коли VHI менше за 35 та більше 30
    all = df[(df['year'] >= first_year) & (df['year'] <= second_year)]['week'].count()
    required = df[(df['year'] >= first_year) & (df['year'] <= second_year) & (df['VHI'] < 35) & (df['VHI'] > 30)]['week'].count()
    percent = (required/all)*100
    print (f'За період з {first_year} по {second_year} було {required} тижнів з помірними посухами.\nЦе дорівнює {percent}% від усієї кількості тижнів цього діапазону')

ExtremesByYear(change_df, 1998.0, 9)
YearswExtremedr (change_df, 2007.0, 2013.0)
YearswModeratedr (change_df, 1991.0, 1999.0)

Мінімальне значення VHI за 1998.0 рік для області під номером 9: 35.03
Максимальне значення VHI за 1998.0 рік для області під номером 9: 68.99
За період з 2007.0 по 2013.0 було 52 тижнів з екстримальними посухами.
Це дорівнює 0.5291005291005291% від усієї кількості тижнів цього діапазону
За період з 1991.0 по 1999.0 було 1057 тижнів з помірними посухами.
Це дорівнює 8.718963952816958% від усієї кількості тижнів цього діапазону
